# SL Fidelidad - 01_Load Data

Fuentes disponibles:
- caracteristicas
- abonos
- eventos
- aspo

In [ ]:
# !pip3 install openpyxl
# !pip3 install xlrd
!which python

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

import pickle

import xlrd
from openpyxl import load_workbook

from helper_functions import str_rem_empty
from helper_functions import info_evento

from functools import reduce

In [ ]:
# seteo carpetas
BASE = Path().absolute().parent
RAW = Path(BASE/'raw')
DATA = Path(BASE/'data')

## Carga de datos y preparacion de tablas

#### Tablon de socios

In [ ]:
# caracteristicas
df_soc = pd.read_csv(RAW/'padronsocios_202101.csv', sep=';')
df_soc.head()

In [ ]:
df_soc.shape

In [ ]:
df_soc.columns

In [ ]:
df_soc['Provincia'].value_counts()

In [ ]:
df_soc['CAT'].value_counts()

In [ ]:
# tipo de socio
df_soc['tipo_socio'] = 'SIMPLE'
df_soc.loc[(df_soc['CAT'] == 'INTERIOR'), 'tipo_socio'] = 'INTERIOR'
df_soc.loc[(df_soc['CAT'] == 'EXTERIOR'), 'tipo_socio'] = 'EXTERIOR'
df_soc.loc[(df_soc['CAT'] == 'PATRIM'), 'tipo_socio'] = 'PATRIMONIAL'
df_soc.loc[(df_soc['CAT'] == 'PLENO'), 'tipo_socio'] = 'PLENO'
df_soc.loc[(df_soc['CAT'] == 'VITALICIO'), 'tipo_socio'] = 'VITALICIO'
df_soc['tipo_socio'].value_counts()

In [ ]:
# forma de pago
df_soc['id_tarjeta'].value_counts()

In [ ]:
df_soc.loc[(df_soc['id_tarjeta'] != 0), 'forma_pago'] = 'DEBITO'
df_soc.loc[(df_soc['id_tarjeta'] == 0), 'forma_pago'] = 'EFECTIVO'
df_soc['forma_pago'].value_counts()

In [ ]:
# refundador
df_soc['Refu'].value_counts()

In [ ]:
df_soc['refunda'] = 0
df_soc.loc[(-df_soc['Refu'].isna()), 'refunda'] = 1
df_soc['refunda'].value_counts()

In [ ]:
# aspo
df_soc['Ult. Cuota Paga'].value_counts()

In [ ]:
df_soc['aspo_dia'] = 0
df_soc.loc[df_soc['Ult. Cuota Paga'].str.slice(start=-4)=='2021', 'aspo_dia'] = 1
df_soc['aspo_dia'].value_counts()

In [ ]:
# renombro
df_soc = df_soc.rename(columns={'Alta': 'fecha_ingreso', 'Fecha Nac.': 'fecha_nac', 'Nro soc': 'socio'})

In [ ]:
df_socios = df_soc[['socio', 'Sexo', 'tipo_socio', 'forma_pago', 'fecha_ingreso', \
                   'refunda', 'fecha_nac', 'aspo_dia']]
df_socios.columns = [x.lower() for x in df_socios.columns]

In [ ]:
df_socios.head()

In [ ]:
df_socios.dtypes

In [ ]:
# index a string
df_socios = df_socios.loc[-df_socios['socio'].isna()]
df_socios['socio'] = df_socios['socio'].astype(int).astype(str)
df_socios = df_socios.set_index('socio')

In [ ]:
df_socios.head()

#### Abonados

In [ ]:
# abonos
df_abo = pd.read_csv(RAW/'abonos_historico.csv', sep=';', encoding = "ISO-8859-1")
# df_abo.head()

In [ ]:
# df_abo.shape

In [ ]:
# año del abono
df_abo['abo_year'] = df_abo['Vencimiento'].str.slice(start = -4)
# df_abo['abo_year'].value_counts()

In [ ]:
# cambio de nombre de vars
df_abo = df_abo.rename(columns={'Socio': 'socio', 'Apellido y Nombre': 'nombre'})

In [ ]:
# elimino espacios vacios en ultimo caracter de nombre
df_abo['nombre'] = [str_rem_empty(x) for x in list(df_abo['nombre'])]

In [ ]:
# pivoteo socio y nombre
socio_nombre = df_abo.loc[-df_abo['socio'].isna(), ['socio', 'nombre']]

# elimino la columna socio con NaN
df_abo = df_abo.drop(['socio'], axis=1)

In [ ]:
# merge
df_abo = pd.merge(df_abo, socio_nombre, how='left', on='nombre')

In [ ]:
# filtro abonos que no importan para el analisis
df_abo = df_abo[df_abo['Sector']!='ESTACIONAMIENTO']
# df_abo['Sector'].value_counts()

In [ ]:
# chequeo nulls
print(sum(df_abo['socio'].isna()))
# df_abo.loc[df_abo['socio'].isna(), 'nombre']

In [ ]:
# elimino los nulls que quedaron y casteo a str
df_abo = df_abo.loc[-df_abo['socio'].isna()]
df_abo['socio'] = df_abo['socio'].astype(int).astype(str)

In [ ]:
# df_abo.shape

In [ ]:
# len(df_abo['socio'].unique())

In [ ]:
# genero dataframe con variables por abono
dfs = [
    # bideg 2020
    pd.DataFrame(
        {'socio':df_abo.loc[(df_abo['abo_year'] == '2020') & (df_abo['Sector'].isin(['PLATEA SUR', 'PLATEA NORTE']))\
                              , 'socio'].unique()
         ,'abo_bid_2020':1}),
    # bideg 2021
    pd.DataFrame(
        {'socio':df_abo.loc[(df_abo['abo_year'] == '2021') & (df_abo['Sector'].isin(['PLATEA SUR', 'PLATEA NORTE']))\
                              , 'socio'].unique()
         ,'abo_bid_2021':1}),
    # poli 2020
    pd.DataFrame(
        {'socio':df_abo.loc[(df_abo['abo_year'] == '2020') & (df_abo['Sector']=='POLIDEPORTIVO'), 'socio'].unique()
         ,'abo_pol_2020':1}),
    # poli 2021
    pd.DataFrame(
        {'socio':df_abo.loc[(df_abo['abo_year'] == '2021') & (df_abo['Sector']=='POLIDEPORTIVO'), 'socio'].unique()
         ,'abo_pol_2021':1})
    ]
df_abonos = reduce(lambda left,right: pd.merge(left,right,on=['socio'],
                                            how='outer'), dfs)

In [ ]:
# df_abonos.shape
# df_abonos.head()

In [ ]:
# df_abonos.sum(axis=1).value_counts()

In [ ]:
df_abonos.columns

In [ ]:
df_abonos = df_abonos.set_index('socio')

#### Eventos

In [ ]:
# eventos
eve = load_workbook(filename = RAW/'eventos.xlsx')
eve_sheets = eve.sheetnames

In [ ]:
print(eve_sheets)
eve_sheets.remove('Abonado')
eve_sheets.remove('UNOxSOCIO')

In [ ]:
# ejemplo
# info_evento(eve_sheets[0])

In [ ]:
# corro la funcion para todas las solapas y almaceno los resultados en un diccionario
eventos = {}
for eve in range(len(eve_sheets)):
    eventos[eve] = info_evento(eve_sheets[eve])

In [ ]:
# print(len(eventos))
# print(type(eventos[0]))
# print(eventos[0].keys())

In [ ]:
# publico promedio
np.mean(np.array([int(eventos[x]['publico']) for x in eventos]))

In [ ]:
# socios totales que asistieron a algun evento
socios_eve = []
for eve in eventos:
    socios_eve.extend(list(eventos[eve]['socios']))
# print(len(socios_eve))
socios_eve = list(set(socios_eve))
# print(len(socios_eve))

In [ ]:
# genero un data frame con la # de eventos por socio por año
socios_eventos = pd.DataFrame({'socio': socios_eve
                              ,'eve_2018': np.zeros(len(socios_eve))
                              ,'eve_2019': np.zeros(len(socios_eve))
                              ,'eve_2020': np.zeros(len(socios_eve))})

In [ ]:
# cantidad de partidos asistidos por socio por año
for socio in range(len(socios_eventos['socio'])):
    for evento in eventos:
        if eventos[evento]['fecha'].year == 2018:
            socios_eventos['eve_2018'][socio] += (socios_eventos['socio'][socio] in eventos[evento]['socios'])
        if eventos[evento]['fecha'].year == 2019:
            socios_eventos['eve_2019'][socio] += (socios_eventos['socio'][socio] in eventos[evento]['socios'])  
        if eventos[evento]['fecha'].year == 2020:
            socios_eventos['eve_2020'][socio] += (socios_eventos['socio'][socio] in eventos[evento]['socios'])

In [ ]:
socios_eventos['socio'] = socios_eventos['socio'].astype(int).astype(str)
df_eventos = socios_eventos.set_index('socio')
# df_eventos.head()

## Salvo las bases

In [ ]:
# socios
pickle.dump( df_socios, open( DATA/"df_socios.p", "wb" ) )
# abonos
pickle.dump( df_abonos, open( DATA/"df_abonos.p", "wb" ) )
# eventos
pickle.dump( df_eventos, open( DATA/"df_eventos.p", "wb" ) )